In [3]:
pip install transformers torch requests beautifulsoup4 pandas nltk feedparser tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import feedparser
import pandas as pd
from transformers import pipeline
from datetime import datetime

# ─── CONFIG ────────────────────────────────────────────────────────────────
STOCK       = "POWERGRID"    # stock ticker or name to search
MAX_HEAD    = 50           # how many headlines to fetch
MODEL_NAME  = "ProsusAI/finbert"
# ────────────────────────────────────────────────────────────────────────────

# 1) Fetch Google News RSS for the stock
def fetch_google_news_headlines(query, max_items=10):
    """
    Uses Google News RSS to return the top headlines for a search query.
    """
    # Google News RSS URL for search:
    url = f"https://news.google.com/rss/search?q={query}+stock+india&hl=en-IN&gl=IN&ceid=IN:en"
    feed = feedparser.parse(url)
    headlines = []
    for entry in feed.entries[:max_items]:
        headlines.append(entry.title)
    return headlines

# 2) Load the FinBERT sentiment-analysis pipeline
sentiment_model = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME
)

# 3) Score each headline
def score_headlines(headlines):
    """
    Returns a DataFrame with [timestamp, headline, label, score].
    """
    results = sentiment_model(headlines)
    df = pd.DataFrame(results)
    df["headline"]  = headlines
    df["timestamp"] = datetime.now()
    # Ensure consistent column names
    df = df.rename(columns={
        df.columns[0]:"label",
        df.columns[1]:"score"
    })
    return df[["timestamp","headline","label","score"]]

# 4) Aggregate into a single weighted score
def aggregate_score(df):
    """
    Map labels to numerical values and compute a weighted mean.
    """
    mapping = {"positive":1, "neutral":0, "negative":-1}
    df["numeric"] = df["label"].map(mapping) * df["score"]
    return df["numeric"].mean()

# 5) End-to-end
def sentiment_bot(stock):
    print(f"\n🔎 Fetching top {MAX_HEAD} headlines for '{stock}' from Google News…")
    headlines = fetch_google_news_headlines(stock, max_items=MAX_HEAD)
    if not headlines:
        print("⚠️  No headlines found.")
        return
    
    print("📰 Headlines:")
    for i, h in enumerate(headlines, 1):
        print(f"  {i}. {h}")
    
    df_scores = score_headlines(headlines)
    avg = aggregate_score(df_scores)
    
    print("\n💬 Sentiment scores:")
    print(df_scores.to_string(index=False))
    print(f"\n👉 Average weighted sentiment: {avg:.3f}")
    
    if avg >  0.05:
        print("Overall sentiment: 📈 POSITIVE → consider LONG bias")
    elif avg < -0.05:
        print("Overall sentiment: 📉 NEGATIVE → consider SHORT bias")
    else:
        print("Overall sentiment: ➖ NEUTRAL")

if __name__ == "__main__":
    sentiment_bot(STOCK)


Device set to use mps:0



🔎 Fetching top 50 headlines for 'POWERGRID' from Google News…
📰 Headlines:
  1. Power Grid Q1 Results: Cons profit falls 2.5% YoY to Rs 3,630 crore, revenue up 2% - The Economic Times
  2. Is Power Grid Corporation of India Limited (POWERGRID) a Good Long Term Bet - Build your wealth with consistent stock growth - Jammu Links News
  3. Stocks to Watch: M&M, IndiGo, Tata Steel, Tata Motors, PowerGrid, and more - Mint
  4. Power Grid Corporation of India Shows Strong Performance Amid Market Volatility - MarketsMojo
  5. Power Grid Corporation of India Limited (POWERGRID) Quarterly Results Are Out - Triple-digit profit margins - Autocar Professional
  6. Power Grid Corporation of India Share Price Live Today - CNBC TV18
  7. Stock Alert: Power Grid Corp, Cipla, HFCL, Central Bank of India - Business Standard
  8. Top stocks to buy or sell today - Saregama, Jio Fin & Power Grid - check stock recommendations for April - The Times of India
  9. 3 PSU Stocks FIIs have piled into. All from th

In [3]:
import feedparser
import pandas as pd
from transformers import pipeline
from datetime import datetime

# ─── CONFIG ────────────────────────────────────────────────────────────────
# List of Nifty 50 tickers (Yahoo Finance format)
nifty50 = [
    'ADANIPORTS.NS','ASIANPAINT.NS','AXISBANK.NS','BAJAJ-AUTO.NS','BAJFINANCE.NS',
    'BAJAJFINSV.NS','BPCL.NS','BHARTIARTL.NS','BRITANNIA.NS','CIPLA.NS',
    'COALINDIA.NS','DIVISLAB.NS','DRREDDY.NS','EICHERMOT.NS','GAIL.NS',
    'GRASIM.NS','HDFCBANK.NS','HINDALCO.NS','HINDUNILVR.NS','HDFC.NS',
    'HCLTECH.NS','ICICIBANK.NS','INDUSINDBK.NS','INFY.NS','ITC.NS',
    'JSWSTEEL.NS','KOTAKBANK.NS','LT.NS','M&M.NS','MARUTI.NS',
    'NESTLEIND.NS','NTPC.NS','ONGC.NS','POWERGRID.NS','SBILIFE.NS',
    'SBIN.NS','SUNPHARMA.NS','TATAMOTORS.NS','TATASTEEL.NS','TCS.NS',
    'TECHM.NS','TITAN.NS','ULTRACEMCO.NS','UPL.NS','WIPRO.NS'
]
MAX_HEADLINES = 10
MODEL_NAME = "ProsusAI/finbert"
# ────────────────────────────────────────────────────────────────────────────

# Load FinBERT sentiment-analysis pipeline
sentiment_model = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME
)

def fetch_google_news_headlines(query, max_items=MAX_HEADLINES):
    """
    Fetch top headlines for a query via Google News RSS.
    """
    url = f"https://news.google.com/rss/search?q={query}+stock+india&hl=en-IN&gl=IN&ceid=IN:en"
    feed = feedparser.parse(url)
    return [entry.title for entry in feed.entries[:max_items]]

def score_headlines(headlines):
    """
    Score headlines using FinBERT transformer.
    Returns DataFrame with columns [timestamp, headline, label, score].
    """
    if not headlines:
        return pd.DataFrame(columns=["timestamp","headline","label","score"])
    results = sentiment_model(headlines)
    df = pd.DataFrame(results)
    df["headline"] = headlines
    df["timestamp"] = datetime.now()
    # rename columns to standard names
    df = df.rename(columns={df.columns[0]:"label", df.columns[1]:"score"})
    return df[["timestamp","headline","label","score"]]

def aggregate_score(df_scores):
    """
    Compute average weighted sentiment: positive=1, neutral=0, negative=-1 times score.
    """
    if df_scores.empty:
        return 0.0
    mapping = {"positive":1, "neutral":0, "negative":-1}
    df_scores["numeric"] = df_scores["label"].map(mapping) * df_scores["score"]
    return df_scores["numeric"].mean()

# Main loop: process all Nifty 50
records = []
for ticker in nifty50:
    query = ticker.split(".")[0]
    headlines = fetch_google_news_headlines(query)
    df_scores = score_headlines(headlines)
    avg = aggregate_score(df_scores)
    bias = "LONG" if avg > 0.05 else "SHORT" if avg < -0.05 else "NEUTRAL"
    records.append({
        "symbol": ticker,
        "avg_score": avg,
        "bias": bias,
        "num_headlines": len(headlines)
    })

# Create final DataFrame
result_df = pd.DataFrame(records)
result_df


Device set to use mps:0


,symbol,avg_score,bias,num_headlines
0,ADANIPORTS.NS,0.091950,LONG,10
1,ASIANPAINT.NS,0.092067,LONG,10
2,AXISBANK.NS,-0.430280,SHORT,10
3,BAJAJ-AUTO.NS,-0.106885,SHORT,10
4,BAJFINANCE.NS,0.007898,NEUTRAL,10
5,BAJAJFINSV.NS,-0.078669,SHORT,10
6,BPCL.NS,-0.019074,NEUTRAL,10
7,BHARTIARTL.NS,-0.086576,SHORT,10
8,BRITANNIA.NS,-0.087223,SHORT,10
9,CIPLA.NS,0.177564,LONG,10
